In [1]:
import pandas as pd
import json
import os
import ipdb
import numpy as np

In [2]:
config_file_path = input()
config_file = open(config_file_path)
config = json.load(config_file)

In [3]:
output_folder = os.path.join(config['output_folder'], config['experiment_name'])
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:
def save_df_to_csv(df, output_file):
    # create the folder if not exists yet
    if not os.path.exists(os.path.dirname(output_file)):
        os.makedirs(os.path.dirname(output_file))

    # Write DataFrame to csv file
    df.to_csv(output_file, index=False)

In [5]:
def replicate_coverage(lib, df, on=None, on_lib=None, on_df=None):
    if on is not None:
        lib_len = len(lib[on])
        df_len = len(df[on])
    elif on_lib is not None and on_df is not None:
        lib_len = len(lib[on_lib])
        df_len = len(df[on_df])

    if lib_len is None or df_len is None:
        return None

    return df_len/lib_len

In [6]:
def clean_raw_pairs_file(df):
    # drop the id_ columns
    # and removed empty rows with unique_17 or barcode
    cleaned_df = df.drop(columns=['id_']).dropna(axis=0, subset=['unique_17', 'barcode'])

    return cleaned_df

In [7]:
def clean_raw_pairs_file_2(df):
    # drop the id_ columns
    # and removed empty rows with unique_17 or barcode
    cleaned_df = df.drop(columns=['id_']).dropna(axis=0, subset=['unique_17'])

    return cleaned_df

In [8]:
def unique_17_count(df):
    unique_17_counts_df = (
        df["unique_17"]
        .value_counts(sort=True)
        .rename_axis("unique_17")
        .reset_index(name="count")
    )

    return unique_17_counts_df

In [9]:
def unique_17_and_bc_count(df):
    unique_17__and_bc_counts_df = (
        df.value_counts(sort=True)
        .rename_axis(["unique_17", "barcode"])
        .reset_index(name="count")
    )

    return unique_17__and_bc_counts_df

In [10]:
def reads_per_million_norm(df, scalling=1e6):
    norm_df = df.copy()
    for column in norm_df:
        total_read_num = df[column].sum()
        scalling_factor = total_read_num / scalling
        norm_df[column] = norm_df[column] / scalling_factor

    return norm_df

In [11]:
def num_of_read_by_fastq(fastq_file):
    with open(fastq_file, 'r') as f:
        x = len(f.readlines())
        
    # 4 lines per read
    return int(x/4)

In [13]:
def read_experimental_dfs(config):
    exp_obj = config.copy()

    for group in exp_obj["groups"]:

        for bio_rep in group["biological_replicates"]:
            
            for tech_rep in bio_rep["technical_replicates"]:
                paired_file = tech_rep["paired_file"]

                df = pd.read_csv(paired_file)
                cleaned_df = clean_raw_pairs_file(df)
                tech_rep["df"] =  cleaned_df

    return exp_obj

In [14]:
lib_csv_path = config["lib_csv_path"]
lib_df = pd.read_csv(lib_csv_path)

In [15]:
def run_experiment_coverage(exp_obj):
    coverage_file = os.path.join(output_folder, "coverage.txt")
    for group in exp_obj["groups"]:
        group_name = group["name"]
        for bio_rep in group["biological_replicates"]:
            bio_name = bio_rep["name"]
            for tech_rep in bio_rep["technical_replicates"]:
                tech_name = tech_rep["name"]

                fastq_file = tech_rep["fastq_file"]
                clipped_fastq_file = tech_rep["clipped_fastq_file"]
                unclipped_fastq_file = tech_rep["unclipped_fastq_file"]
                too_long_fastq_file = tech_rep["too_long_fastq_file"]
                too_short_fastq_file = tech_rep["too_short_fastq_file"]

                unique_17_count_df = unique_17_count(tech_rep["df"])

                coverage = replicate_coverage(
                    lib_df, unique_17_count_df, on_lib="unique_17", on_df="unique_17"
                )
                fastq_reads = num_of_read_by_fastq(fastq_file)
                clipped_fastq_reads = num_of_read_by_fastq(clipped_fastq_file)
                unclipped_fastq_reads = num_of_read_by_fastq(unclipped_fastq_file)
                too_long_fastq_reads = num_of_read_by_fastq(too_long_fastq_file)
                too_short_fastq_reads = num_of_read_by_fastq(too_short_fastq_file)

                output_line = (
                    f"{group_name} -> {bio_name} -> {tech_name}:"
                    f"\n\tnumber of aligned reads: {tech_rep['df'].shape[0]} ===> coverage: {coverage}"
                    f"\n\tfastq number of reads: {fastq_reads}"
                    f"\n\tclipped fastq number of reads: {clipped_fastq_reads} ===> clipped reads coverage: {clipped_fastq_reads/fastq_reads}"
                    f"\n\tunclipped fastq number of reads: {unclipped_fastq_reads} ===> unclipped reads coverage: {unclipped_fastq_reads/fastq_reads}"
                    f"\n\ttoo long fastq number of reads: {too_long_fastq_reads} ===> too long reads coverage: {too_long_fastq_reads/fastq_reads}"
                    f"\n\ttoo short fastq number of reads: {too_short_fastq_reads} ===> too short reads coverage: {too_short_fastq_reads/fastq_reads}"
                )

                print(output_line)
                with open(coverage_file, "a+") as out:
                    out.write(output_line + "\n")


In [16]:
expr_obj = read_experimental_dfs(config)

In [ ]:
run_experiment_coverage(expr_obj)

In [17]:
all_unique_17s = lib_df["unique_17"]
len(all_unique_17s)

6000

In [18]:
def run_experiment_analysis(expr_obj):
    expand_output_df = pd.DataFrame(data={"unique_17": all_unique_17s})
    bio_output_df = pd.DataFrame(data={"unique_17": all_unique_17s})

    for group in expr_obj["groups"]:
        group_name = group["name"]
        for bio_rep in group["biological_replicates"]:
            bio_name = bio_rep["name"]
            bio_count_col_name = f"{group_name}_{bio_name}"
            bio_count_df = pd.DataFrame(data={"unique_17": all_unique_17s, f"{bio_count_col_name}": np.zeros(len(all_unique_17s))})
            for tech_rep in bio_rep["technical_replicates"]:
                tech_name = tech_rep["name"]
                
                unique_17_count_df = unique_17_count(tech_rep["df"])
                tech_count_col_name = f"{group_name}_{bio_name}_{tech_name}"
                unique_17_count_df = unique_17_count_df.rename(columns={"count" : tech_count_col_name})
                bio_count_df = bio_count_df.merge(unique_17_count_df, how="left", on="unique_17").fillna(0)
                bio_count_df[bio_count_col_name] = bio_count_df[bio_count_col_name] + bio_count_df[tech_count_col_name]
            
            bio_output_df = bio_output_df.merge(bio_count_df[["unique_17",bio_count_col_name]], how="left", on="unique_17")
            expand_output_df = expand_output_df.merge(bio_count_df, how="left", on="unique_17")

    return expand_output_df, bio_output_df

In [19]:
expand_output_df, bio_output_df = run_experiment_analysis(expr_obj)

In [20]:
bio_output_df.head()

,unique_17,NCI_R1_bio_1,TREGS_R1_bio_1,NCI_R2_bio_1,TREGS_R2_bio_1
0,ACTACCTGAAGAACCTT,2.0,0.0,0.0,0.0
1,GAGCTAAATGGCTGATT,45.0,12.0,60.0,5.0
2,GTGACCACACTTACAGT,23.0,56.0,23.0,45.0
3,TTGTTGGCGAGCAGTGT,4.0,9.0,2.0,3.0
4,CAATATCGGCGAGCTCT,63.0,28.0,66.0,23.0


In [21]:
expand_output_df.head()

,unique_17,NCI_R1_bio_1,NCI_R1_bio_1_tech_2,NCI_R1_bio_1_tech_3,TREGS_R1_bio_1,TREGS_R1_bio_1_tech_2,TREGS_R1_bio_1_tech_3,NCI_R2_bio_1,NCI_R2_bio_1_tech_2,NCI_R2_bio_1_tech_3,TREGS_R2_bio_1,TREGS_R2_bio_1_tech_2,TREGS_R2_bio_1_tech_3
0,ACTACCTGAAGAACCTT,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GAGCTAAATGGCTGATT,45.0,35.0,10.0,12.0,0.0,12.0,60.0,41.0,19.0,5.0,0.0,5.0
2,GTGACCACACTTACAGT,23.0,14.0,9.0,56.0,41.0,15.0,23.0,16.0,7.0,45.0,36.0,9.0
3,TTGTTGGCGAGCAGTGT,4.0,2.0,2.0,9.0,0.0,9.0,2.0,0.0,2.0,3.0,0.0,3.0
4,CAATATCGGCGAGCTCT,63.0,23.0,40.0,28.0,18.0,10.0,66.0,20.0,46.0,23.0,19.0,4.0


In [24]:
only_tech = expand_output_df.set_index("unique_17").drop(["NCI_R1_bio_1","TREGS_R1_bio_1","NCI_R2_bio_1","TREGS_R2_bio_1"], axis=1)

In [25]:
reads_per_million_norm(only_tech)

,NCI_R1_bio_1_tech_2,NCI_R1_bio_1_tech_3,TREGS_R1_bio_1_tech_2,TREGS_R1_bio_1_tech_3,NCI_R2_bio_1_tech_2,NCI_R2_bio_1_tech_3,TREGS_R2_bio_1_tech_2,TREGS_R2_bio_1_tech_3
unique_17,,,,,,,,
ACTACCTGAAGAACCTT,1.033924,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
GAGCTAAATGGCTGATT,18.093662,6.139022,0.000000,7.461916,24.078922,9.398808,0.000000,5.225862
GTGACCACACTTACAGT,7.237465,5.525120,18.952132,9.327395,9.396653,3.462719,18.291279,9.406551
TTGTTGGCGAGCAGTGT,1.033924,1.227804,0.000000,5.596437,0.000000,0.989348,0.000000,3.135517
CAATATCGGCGAGCTCT,11.890121,24.556087,8.320448,6.218264,11.745816,22.755008,9.653731,4.180689
...,...,...,...,...,...,...,...,...
CCCTAGGGGATAGCGAT,9.305312,11.050239,7.395954,1.243653,7.634780,12.861526,9.653731,3.135517
CCGAACGTTGTTGCATG,94.604005,82.876795,307.394336,289.149254,82.220711,81.126551,276.401557,224.712055
CCTTCGTTTCACTCCTC,3.618732,3.683413,14.329661,11.192874,3.523745,1.978696,9.653731,11.496896


In [54]:
expand_output_df_csv = os.path.join(output_folder, "all_replicates_sum.csv")
save_df_to_csv(expand_output_df, expand_output_df_csv)

In [55]:
bio_output_df_csv = os.path.join(output_folder, "all_bio_replicates_sum.csv")
save_df_to_csv(bio_output_df, bio_output_df_csv)